In [1]:
from datetime import datetime
from dataclasses import dataclass



class Connection:
    def __init__(self, line, departure_time, arrival_time, start_latitude, start_longitude, end_latitude, end_longitude, end_stop):
        self.line: str = line
        self.departure_time: datetime = departure_time
        self.arrival_time: datetime = arrival_time
        self.start_latitude: float = start_latitude
        self.start_longitude: float = start_longitude
        self.end_latitude: float = end_latitude
        self.end_longitude: float = end_longitude
        self.end_stop: str = end_stop
        
    def __repr__(self):
        return f"Connection({self.line}, {self.departure_time.strftime('%H:%M:%S')} -> {self.arrival_time.strftime('%H:%M:%S')}, {self.end_stop})"




class BusStop:
    def __init__(self, name: str):
        self.name: str = name
        self.connections: dict[str, list[Connection]] = {}

    def add_connection(self, end_stop: str, connection: Connection):
        if end_stop not in self.connections:
            self.connections[end_stop] = [connection]
        else:
            self.connections[end_stop].append(connection)
    
    def __repr__(self):
        return f"BusStop({self.name}, {len(self.connections)} connections)"



In [2]:
import pandas as pd
from datetime import datetime

datetime_format = '%Y-%m-%d %H:%M:%S'


def convert_time(time: str) -> datetime:
    hour = int(time[:2])
    if hour >= 24:
        hour -= 24
        return datetime.strptime(f'2025-01-02 {hour:02}{time[2:]}', datetime_format)
    else:
        return datetime.strptime(f'2025-01-01 {time}', datetime_format)


def add_connection(graph: dict[str, BusStop], start_stop: str,
                   end_stop: str, connection: Connection):
    if start_stop not in graph:
        graph[start_stop] = BusStop(name=start_stop)
    if end_stop not in graph:
        graph[end_stop] = BusStop(name=end_stop)
    graph[start_stop].add_connection(end_stop, connection)


def load_csv_data(filename: str):
    df = pd.read_csv(filename)
    graph: dict[str, BusStop] = {}

    for index, row in df.iterrows():
        start_stop = row['start_stop']
        end_stop = row['end_stop']
        departure_time = convert_time(row['departure_time'])
        arrival_time = convert_time(row['arrival_time'])
        start_lat = row['start_stop_lat']
        start_lon = row['start_stop_lon']
        end_lat = row['end_stop_lat']
        end_lon = row['end_stop_lon']
        connection = Connection(
            line=row['line'],
            departure_time=departure_time,
            arrival_time=arrival_time,
            start_latitude=start_lat,
            start_longitude=start_lon,
            end_latitude=end_lat,
            end_longitude=end_lon,
            end_stop=end_stop
        )
        add_connection(graph, start_stop, end_stop, connection)
    
    return graph

In [62]:
graph = load_csv_data('mpk_indexed.csv')

/var/folders/03/n179cc412h58x4qb9mt1bv_m0000gn/T/ipykernel_50637/4144027882.py:26: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [73]:
def find_earliest_connection(connections: list[Connection], current_time: datetime, previous_line=None, transfer_time=2) -> Connection:

    earliest = None
    earliest_departure = None
    
    for conn in connections:
        required_time = current_time
        if previous_line is not None and conn.line != previous_line:
            required_time = current_time + timedelta(minutes=transfer_time)
        
        if conn.departure_time >= required_time and (earliest is None or conn.departure_time < earliest_departure):
            earliest = conn
            earliest_departure = conn.departure_time
    
    return earliest

In [67]:
import heapq
from datetime import timedelta

In [68]:

def dijkstra(start_stop: str, end_stop: str, departure_time: str, graph: dict[str, BusStop], transfer_time: int = 2):

    if start_stop not in graph or end_stop not in graph:
        return None, []

    start_time = convert_time(departure_time)

    distances = {stop: float('inf') for stop in graph}
    previous = {stop: None for stop in graph}
    arrival_times = {stop: None for stop in graph}
    previous_lines = {stop: None for stop in graph}

    distances[start_stop] = 0
    arrival_times[start_stop] = start_time

    priority_queue = [(0, start_stop)]

    visited_nodes = 0
    visited_connections = 0

    while priority_queue:
        current_distance, current_stop = heapq.heappop(priority_queue)

        if current_stop == end_stop:
            break

        if current_distance > distances[current_stop]:
            continue

        visited_nodes += 1 

        current_time = arrival_times[current_stop]
        current_line = previous_lines[current_stop]

        for next_stop, connections in graph[current_stop].connections.items():
            visited_connections += 1  

            earliest_conn = find_earliest_connection(
                connections, 
                current_time, 
                previous_line=current_line, 
                transfer_time=transfer_time
            )

            if earliest_conn is None:
                continue

            travel_time = (earliest_conn.arrival_time - current_time).total_seconds() / 60

            if distances[current_stop] + travel_time < distances[next_stop]:
                distances[next_stop] = distances[current_stop] + travel_time
                previous[next_stop] = (current_stop, earliest_conn)
                arrival_times[next_stop] = earliest_conn.arrival_time
                previous_lines[next_stop] = earliest_conn.line

                heapq.heappush(priority_queue, (distances[next_stop], next_stop))

    print(f"Odwiedzone węzły: {visited_nodes}, odwiedzone krawędzie: {visited_connections}")

    route = []
    current_stop = end_stop
    while current_stop and previous[current_stop]:
        prev_stop, conn = previous[current_stop]
        route.append(conn)
        current_stop = prev_stop

    route.reverse()

    total_time = distances[end_stop] if distances[end_stop] != float('inf') else None
    return total_time, route


In [109]:
start = "Iwiny - rondo"
end = "Psie Pole (Rondo Lotników Polskich)"
departure = "11:25:00"

total_time, route = dijkstra(start, end, departure, graph)
print_path(total_time, route)


Odwiedzone węzły: 400, odwiedzone krawędzie: 1209
Całkowity czas podróży: 58.0 minut

Trasa:
1. Linia 110: odjazd 11:25:00, przyjazd 11:26:00 -> Vivaldiego
2. Linia 110: odjazd 11:26:00, przyjazd 11:28:00 -> Kajdasza
3. Linia 110: odjazd 11:28:00, przyjazd 11:30:00 -> Jagodzińska
4. Linia 110: odjazd 11:30:00, przyjazd 11:31:00 -> Malinowskiego
5. Linia 110: odjazd 11:31:00, przyjazd 11:32:00 -> Konduktorska
6. Linia 110: odjazd 11:32:00, przyjazd 11:33:00 -> Buforowa-Rondo
7. Linia 110: odjazd 11:33:00, przyjazd 11:34:00 -> BARDZKA (Cmentarz)
8. Linia 110: odjazd 11:34:00, przyjazd 11:35:00 -> Morwowa
9. Linia 110: odjazd 11:35:00, przyjazd 11:36:00 -> Krynicka
10. Linia 110: odjazd 11:36:00, przyjazd 11:38:00 -> Bardzka
11. Linia 110: odjazd 11:38:00, przyjazd 11:40:00 -> Kamienna
12. Linia 110: odjazd 11:40:00, przyjazd 11:41:00 -> Prudnicka
13. Linia 110: odjazd 11:41:00, przyjazd 11:42:00 -> Hubska (Dawida)
14. Linia 16 (przesiadka): odjazd 11:47:00, przyjazd 11:51:00 -> Kościuszk

In [200]:
import heapq
from datetime import datetime
from typing import Callable, List, Dict, Optional, Tuple

def astar(
    start_stop: str, 
    end_stop: str, 
    departure_time: str, 
    graph: dict[str, BusStop], 
    heuristic_function: Callable[[str, str, Dict[str, BusStop]], float],
    transfer_time: int = 2
):

    if start_stop not in graph:
        return None, []
    if end_stop not in graph:
        return None, []

    start_time = convert_time(departure_time)

    distances = {stop: float('inf') for stop in graph}
    previous = {stop: None for stop in graph}
    arrival_times = {stop: None for stop in graph}
    previous_lines = {stop: None for stop in graph}

    distances[start_stop] = 0
    arrival_times[start_stop] = start_time

    priority_queue = [(0 + heuristic_function(start_stop, end_stop, graph), start_stop)]

    visited_nodes = 0
    visited_connections = 0

    while priority_queue:
        current_f_score, current_stop = heapq.heappop(priority_queue)


        if current_stop == end_stop:
            break

        current_h_score = heuristic_function(current_stop, end_stop, graph)
        current_g_score = current_f_score - current_h_score
        
        if current_g_score > distances[current_stop]:
            continue

        visited_nodes += 1

        current_time = arrival_times[current_stop]
        current_line = previous_lines[current_stop]

        for next_stop, connections in graph[current_stop].connections.items():
            visited_connections += 1


            earliest_conn = find_earliest_connection(
                connections, 
                current_time, 
                previous_line=current_line, 
                transfer_time=transfer_time
            )

            if earliest_conn is None:
                continue

            travel_time = (earliest_conn.arrival_time - current_time).total_seconds() / 60

            change = False
            if current_line and earliest_conn.line != current_line:
                change = True

            if distances[current_stop] + travel_time < distances[next_stop]:
                distances[next_stop] = distances[current_stop] + travel_time
                previous[next_stop] = (current_stop, earliest_conn)
                arrival_times[next_stop] = earliest_conn.arrival_time
                previous_lines[next_stop] = earliest_conn.line

                next_h_score = heuristic_function(next_stop, end_stop, graph, 
                                                  current_stop=current_stop,
                                                  change=change
                                                  )
                heapq.heappush(priority_queue, (distances[next_stop] + next_h_score, next_stop))


    if distances[end_stop] == float('inf') or previous[end_stop] is None:
        print("Nie znaleziono trasy.")
        return None, []

    route = []
    current_stop = end_stop
    while current_stop and previous[current_stop]:
        prev_stop, conn = previous[current_stop]
        route.append(conn)
        current_stop = prev_stop

    route.reverse()

    total_time = distances[end_stop] if distances[end_stop] != float('inf') else None
    print(f"Odwiedzone węzły: {visited_nodes}, odwiedzone krawędzie: {visited_connections}")
    return total_time, route

def convert_time(time_str: str) -> datetime:
    today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
    time_parts = list(map(int, time_str.split(':')))
    
    if len(time_parts) == 2:
        time_parts.append(0)
        
    return today.replace(hour=time_parts[0], minute=time_parts[1], second=time_parts[2])



In [203]:
def zero_heuristic(next_stop: str, target_stop: str, graph: Dict[str, BusStop], current_stop=None, change=None) -> float:
    return 0

In [ ]:
import math

def extract_first_connection(stop_name: str, graph: Dict[str, BusStop]):
    try:
        for next_stop, connections in graph[stop_name].connections.items():
            if connections: 
                return connections[0]
    except (KeyError, IndexError):
        pass
    return None


def euclidean_distance(x1, x2, y1, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def euclidean_distance_heuristic(next_stop: str, target_stop: str, graph: Dict[str, BusStop],
                                  current_stop=None, change=None) -> float:
    first_next_conn = extract_first_connection(next_stop, graph)
    first_target_conn = extract_first_connection(target_stop, graph)
    
    if first_next_conn is None or first_target_conn is None:
        return 0
    
    dist = euclidean_distance(
        first_next_conn.start_latitude, first_target_conn.start_latitude,
        first_next_conn.start_longitude, first_target_conn.start_longitude
    )
    
    # Konwersja do minut podróży
    # 1 stopień szerokości/długości geograficznej to około 111 km
    # średnia prędkość 30 km/h -> 0.5 km/min
    travel_time_estimate = dist * 111 / 0.5
    
    return travel_time_estimate * 0.8

In [ ]:
def angle_between_heuristic(next_stop: str, 
                            target_stop: str, 
                            graph: Dict[str, BusStop], 
                            current_stop=None, 
                            change=None
                            ) -> float:
    
    try:
    
        first_next_conn = extract_first_connection(next_stop, graph)
        first_target_conn = extract_first_connection(target_stop, graph)
        first_current_conn = extract_first_connection(current_stop, graph)

        curr_target_dist = euclidean_distance(first_current_conn.start_latitude, first_target_conn.start_latitude,
                                            first_current_conn.start_longitude, first_target_conn.start_longitude)
        
        curr_next_dist = euclidean_distance(first_current_conn.start_latitude, first_next_conn.start_latitude,
                                            first_current_conn.start_longitude, first_next_conn.start_longitude)
        
        next_target_dist = euclidean_distance(first_target_conn.start_latitude, first_next_conn.start_latitude,
                                            first_target_conn.start_longitude, first_next_conn.start_longitude)
        

        cos = (next_target_dist**2- curr_target_dist**2 - curr_next_dist**2) / ( -2.0 * curr_target_dist * curr_next_dist)


        if change and cos < 0.5:
            return (2.5 - cos)
        return 0    
    except:
        return 0
    
                                                                         
    
    


In [239]:
start = "Słoneczna"
end = "POŚWIĘTNE"
departure = "11:36:00"

total_time, route = astar(start, end, departure, graph, heuristic_function=angle_between_heuristic)
print_path(total_time, route)


Odwiedzone węzły: 131, odwiedzone krawędzie: 443
Całkowity czas podróży: 62.0 minut

Trasa:
1. Linia 133: odjazd 11:44:00, przyjazd 11:45:00 -> Marcepanowa
2. Linia 133: odjazd 11:45:00, przyjazd 11:46:00 -> WAŁBRZYSKA
3. Linia 133: odjazd 11:46:00, przyjazd 11:47:00 -> Kościelna
4. Linia 133: odjazd 11:47:00, przyjazd 11:48:00 -> KLECINA
5. Linia 133: odjazd 11:48:00, przyjazd 11:49:00 -> Skarbowców
6. Linia 133: odjazd 11:49:00, przyjazd 11:50:00 -> Os. Przyjaźni
7. Linia 133: odjazd 11:50:00, przyjazd 11:51:00 -> Zimowa
8. Linia 133: odjazd 11:51:00, przyjazd 11:52:00 -> Sowia
9. Linia 133: odjazd 11:52:00, przyjazd 11:53:00 -> Chłodna
10. Linia 133: odjazd 11:53:00, przyjazd 11:54:00 -> Wawrzyniaka
11. Linia 133: odjazd 11:54:00, przyjazd 11:55:00 -> Modlińska
12. Linia 133: odjazd 11:55:00, przyjazd 11:57:00 -> Gajowicka (szkoła)
13. Linia 133: odjazd 11:57:00, przyjazd 11:59:00 -> Gajowicka
14. Linia 133: odjazd 11:59:00, przyjazd 12:01:00 -> Hallera
15. Linia 6 (przesiadka): odj